In [58]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV
from project_heart.enums import *

In [59]:
dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
files = os.listdir(dir)

file_id = 3 # 5 has a problem

print("File:", files[file_id])

lv = LV()
lv.from_pyvista_read(dir/files[file_id], identifier="elemTag", threshold=[0, 1])

File: Full_Heart_Mesh_12.vtk


In [60]:
lv.identify_surfaces(
  
  ab_ql=0.03, 
  ab_qh=0.70,
  
  alpha_atr=0.20,  # coeff for radial distance computation
  alpha_mtr=0.69,
  
  beta_atr=0.15,  # coeff for second radial distance computation
  beta_mtr=0.27,
  
  gamma_atr=89,
  gamma2_mtr=25,
  
  phi_atr=79,
  epi_angle=100
)

plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node("MITRAL"), color="red", point_size=300)
plotter.add_points(lv.get_virtual_node("AORTIC"), color="purple", point_size=300)
plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [61]:
lv.mitral_info

{'R': 16.19505571655343, 'C': array([-1.5481617 , 25.68176791, 11.36415558])}

In [62]:
lv.aortic_info

{'R': 14.923538134565778, 'C': array([-7.5238643 ,  3.63141074, 21.63932695])}

In [63]:
def generate_circle_by_vectors(t, C, r, n, u):
    n = n/np.linalg.norm(n)
    u = u/np.linalg.norm(u)
    P_circle = r*np.cos(t)[:,np.newaxis]*u + r*np.sin(t)[:,np.newaxis]*np.cross(n,u) + C
    return P_circle

In [64]:
def fit_plane(points):
  P_mean = points.mean(axis=0)
  P_centered = points - P_mean
  U,s,V = np.linalg.svd(P_centered)
  normal = V[2,:]
  d = -np.dot(P_mean, normal) 
  return normal, d

In [65]:
def create_rim_circunference(
    center, radius, height, normal, cross_normal,
    radial_resolution=64, 
    # longitudinal_resolution=2,
  ):

  # step_h = height/longitudinal_resolution
  c1 = center - normal*height*0.5
  c2 = center + normal*height*0.5

  rim_angles = np.linspace(0, 2*np.pi, radial_resolution+1)[:-1]
  rim = generate_circle_by_vectors(rim_angles, c1, radius,
                                   normal, cross_normal)
  
  
  rim_2 = np.copy(rim)
  for i, node in enumerate(rim):
    rim_2[i] = node + height*normal
  
  # set rim center as mean from two centers
  rim_center_2 = c2#center + step_h*normal
  rim_center = np.mean([center, rim_center_2], axis=0)
  
  # set elements
  elements = []
  n_nodes = len(rim)
  for i, j in zip(range(0, n_nodes), range(n_nodes, n_nodes*2-1)):
    elements.append((
        i, j, j+1, i+1
    ))
  
  rim = np.vstack([rim, rim_2])
  return rim, rim_center, elements

In [66]:
import scipy
def query_kdtree(A, B):
  """
    Performs a kd-query of B on A; in which A is the kd-tree.
    Returns the indexes of closest points of B w.r.t. A.
  """
  tree = scipy.spatial.cKDTree(A)
  dist, indexes = tree.query(B)
  return indexes

In [67]:
def set_spring_indexes(A, B):
  from_nodes = np.arange(len(A)).reshape(-1,1)
  to_nodes = query_kdtree(B, A).reshape(-1,1)
  return np.hstack((from_nodes, to_nodes))

In [68]:
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points)-1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points)-1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    # poly = poly.tube(radius=0.1)
    return poly

In [69]:
from collections import deque
def get_springs_pts_for_plot(geo_pts, rim_pts, s_idx_map, color="cyan", n_skip=1):
  pts_a = geo_pts[s_idx_map[:, 0]][::n_skip]
  pts_b = rim_pts[s_idx_map[:, 1]][::n_skip]
  lines = None
  for a, b in zip(pts_a, pts_b):
    if lines is None:
      lines = lines_from_points(np.array([a,b]))
    else:
      lines = lines.merge(lines_from_points(np.array([a,b])))  
  return lines

In [70]:
atr_pts = lv.points(mask=lv.get_nodeset("AORTIC"))
mtr_pts = lv.points(mask=lv.get_nodeset("MITRAL"))

In [71]:
lvnormal = lv.get_normal()

In [72]:
n, _ = fit_plane(atr_pts)
n = -n if n[2] < 0 else n
x = np.cross(n, lv._Z)
c = lv.get_virtual_node(LV_VIRTUAL_NODES.AORTIC.value) + n*10
r = lv.aortic_info["R"] * 0.70
h = 2
atr_rim, rim_center, rim_el = create_rim_circunference(c, r, h, -n, x)
atr_rim_string_map = set_spring_indexes(atr_pts, atr_rim)
atr_rim_string_lines = get_springs_pts_for_plot(atr_pts, atr_rim, atr_rim_string_map, n_skip=1)

In [73]:
# def check_rim_neighbors(string_map):
#   sort_idxs = string_map[:, 1].argsort()
#   vals = string_map[sort_idxs][:, 1]
#   to_exclude = np.zeros(len(vals), dtype=bool)
#   for i in range(len(vals) - 1):
#     if vals[i+1] != vals[i]:
#       to_exclude[i+1] = True
#   if vals[-1] != vals[-2]:
#     to_exclude[-1] = True
#   # print(to_exclude)
#   return string_map[~to_exclude]

In [74]:
n, _ = fit_plane(mtr_pts)
n = -n if n[2] < 0 else n
x = np.cross(n, lv._Z)
c = lv.get_virtual_node(LV_VIRTUAL_NODES.MITRAL.value) + n*10
r = lv.mitral_info["R"] * 0.70
h = 2
mtr_rim, rim_center, rim_el = create_rim_circunference(c, r, h, -n, x)
mtr_rim_string_map = set_spring_indexes(mtr_pts, mtr_rim)
# mtr_rim_string_map = check_rim_neighbors(mtr_rim_string_map)
mtr_rim_string_lines = get_springs_pts_for_plot(mtr_pts, mtr_rim, mtr_rim_string_map, n_skip=1)

In [78]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node("MITRAL"), color="green", point_size=300)
plotter.add_points(lv.get_virtual_node("AORTIC"), color="red", point_size=300)
plotter.add_points(atr_rim, color="red", point_size=300)
plotter.add_points(mtr_rim, color="green", point_size=300)
plotter.add_mesh(atr_rim_string_lines, color="red", opacity=0.5, show_edges=False)
plotter.add_mesh(mtr_rim_string_lines, color="green", opacity=0.5, show_edges=False)
plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…